#### This was a communicator for all the category products which I didn't already extract for. which DOES NOT WORK.

In [1]:
import sqlite3
import json
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import SQLDatabase
from dotenv import load_dotenv, find_dotenv
import sys
import stat
from langchain_community.llms import HuggingFaceEndpoint
from langchain_anthropic import ChatAnthropic

c:\Users\tomce\miniconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(find_dotenv())
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [5]:
# Function to create database and tables
def create_database():
    conn = sqlite3.connect('product_database_universal.db')
    c = conn.cursor()

    # Create products table
    c.execute('''CREATE TABLE IF NOT EXISTS products (
                    id INTEGER PRIMARY KEY,
                    url TEXT,
                    title TEXT,
                    warranty INTEGER,
                    regular_price REAL,
                    happy_price REAL,
                    category TEXT
                 )''')

    # Create feature_products table
    c.execute('''CREATE TABLE IF NOT EXISTS feature_products (
                    id INTEGER PRIMARY KEY,
                    feature_name TEXT,
                    feature_value TEXT
                 )''')

    # Create product_feature table
    c.execute('''CREATE TABLE IF NOT EXISTS product_feature (
                    product_id INTEGER,
                    feature_id INTEGER,
                    FOREIGN KEY (product_id) REFERENCES products(id),
                    FOREIGN KEY (feature_id) REFERENCES feature_products(id)
                 )''')

    conn.commit()
    conn.close()

# Function to insert product data into database
def insert_product_data(product_data):
    conn = sqlite3.connect('product_database_universal.db')
    c = conn.cursor()

    # Insert product data into products table
    c.execute('''INSERT INTO products (url, title, warranty, regular_price, happy_price, category)
                 VALUES (?, ?, ?, ?, ?, ?)''', (
                    product_data.get('url'),
                    product_data.get('title'),
                    product_data.get('warranty_months'),
                    product_data.get('regular_price_mkd'),
                    product_data.get('happy_price_mkd'),
                    product_data.get('category')
                 ))
    product_id = c.lastrowid

    # Insert feature data into feature_gaming_laptops table and product_feature table
    for feature, value in product_data.get('features', {}).items():
        # Check if feature already exists
        c.execute('''SELECT id FROM feature_products WHERE feature_name = ? AND feature_value = ?''', (feature, value))
        feature_row = c.fetchone()
        if feature_row:
            feature_id = feature_row[0]
        else:
            c.execute('''INSERT INTO feature_products (feature_name, feature_value) VALUES (?, ?)''', (feature, value))
            feature_id = c.lastrowid

        # Insert into product_feature table
        c.execute('''INSERT INTO product_feature (product_id, feature_id) VALUES (?, ?)''',
                  (product_id, feature_id))

    conn.commit()
    conn.close()

# Function to read JSON files from a directory and insert data into database
def process_json_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            with open(os.path.join(directory, filename), 'r') as file:
                product_data = json.load(file)
                insert_product_data(product_data)

# Create database and tables
create_database()

# Process JSON files and insert data into database
json_directory = r'C:\Users\tomce\OneDrive - UKIM, FINKI\Desktop\Fakultet 3ta Godina\2 Sesti Semestar\0 DATA SCIENCE SEMINARSKA\1 Starting Over\products_categories_fixed\all_extracted_universal_template'
process_json_files(json_directory)


In [3]:
def get_features_string(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Query to select all features
    cursor.execute("SELECT feature_name, feature_value FROM feature_products WHERE feature_value IS NOT NULL")
    
    # Fetch all rows from the executed query
    features = cursor.fetchall()
    
    # Close the database connection
    conn.close()
    
    # Format the features into the desired string format
    features_string = ", ".join([f"{name}={value}" for name, value in features])
    
    return features_string

# Example usage
db_path = 'product_database_universal.db'
features_string = get_features_string(db_path)
print(features_string)

printing_volume_x_mm=200, printing_volume_y_mm=200, printing_volume_z_mm=200, printing_speed_mm_s=100, nozzle_temperature_max_celsius=260, bed_temperature_max_celsius=120, filament_diameter_mm=1.75, nozzle_diameter_mm=0.4, supported_file_formats=STL, 3mf, .stl, .obj, G-code, printing_technology=FDM, resolution=(1920x1080), sensor_size=1/2.9”, format=MJPG, YUY2, compatibility=7/8.1/10, fps=30, display_size_inches=10.4, resolution=2000 x 1200 pixels, processor=MediaTek: MT8183, ram_gb=4, storage_gb=128, primary_camera_mp=8, secondary_camera_mp=5, connection_type=USB, cable_length_meters=1.5, connection_type=Wi-Fi, IEEE 802.11 b/g/n, voltage=AC 100–240 V, frequency=50/60 Hz, cable_length_meters=3, frequency_range_ghz=2.412—2.484 GHz, max_power_w=2400, outlet_count=3, input_voltage=AC 100–240 V, 50/60 Hz, maximum_power_watts=2400, outlet_count=6, app_name=Smart Life, compatibility=Android 4.1,iOS 9.0, usb_port_count=4, dimensions_mm=320x270x3, dimensions_mm=350x250x1, dimensions_mm=300x700

In [4]:
conn = sqlite3.connect('product_database_universal.db')
c = conn.cursor()
input_db = SQLDatabase.from_uri('sqlite:///product_database_universal.db')

In [18]:
template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Please note that in the schema, the regular_price is the normal price, while the happy_price is a discounted one.
Table Descriptions:

products Table:
This table stores general information about products.
Columns:
id: Unique identifier for each product.
url: URL link to the product page.
title: Title or name of the product.
warranty: Duration of warranty in months.
regular_price: Normal price of the product.
happy_price: Discounted price (if applicable).
category: Category of the product (e.g., “laptops,” “phones,” etc.).

feature_products Table:
This table captures individual features of products.
Columns:
id: Unique identifier for each feature.
feature_name: Name of the feature (e.g., “processor type,” “RAM size”).
feature_value: Value associated with the feature (e.g., “Intel Core i7,” “16GB”).

product_feature Table:
This table establishes a many-to-many relationship between products and their features.
Columns:
product_id: Foreign key referencing the id column in the products table.
feature_id: Foreign key referencing the id column in the feature_products table.


You ABSOLUTELY MUST use the ‘LIKE’ operator instead of ‘=’ in the SQL query for columns in the tables that are of TEXT value.
You ABSOLUTELY MUST use the ‘=’ operator instead of ‘LIKE’ in the SQL query for columns in the tables that are of REAL, DOUBLE, or INTEGER value.

Question: {question}
SQL Query:
"""
prompt = ChatPromptTemplate.from_template(template)

def get_schema(db):
    schema = input_db.get_table_info()
    return schema

In [19]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=128, temperature=0.1, token=HUGGINGFACEHUB_API_TOKEN
)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\tomce\.cache\huggingface\token
Login successful


In [11]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.1)

In [ ]:
llm = ChatAnthropic(temperature=0, model_name="claude-3-sonnet-20240229")

In [20]:
sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

user_question = 'What headphones are there?'
smth= sql_chain.invoke({"features": features_string,"question": user_question})
print(smth)
output_string = smth.replace("```sql", "").replace("```", "")
print(output_string)


```sql
SELECT products.title
FROM products
JOIN product_feature ON products.id = product_feature.product_id
JOIN feature_products ON product_feature.feature_id = feature_products.id
WHERE feature_products.feature_name LIKE '%headphones%';
```

This query will return the titles of all products that have the word "headphones" in their associated features.


SELECT products.title
FROM products
JOIN product_feature ON products.id = product_feature.product_id
JOIN feature_products ON product_feature.feature_id = feature_products.id
WHERE feature_products.feature_name LIKE '%headphones%';


This query will return the titles of all products that have the word "headphones" in their associated features.


#### DOBRO QUERRY OD MISTRAL INSTRUCT 0.3

In [28]:
sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

user_question = 'What gaming laptops are there that have a 1920x1080 display resolution and a 1TB SSD storage?'
smth= sql_chain.invoke({"features": features_string,"question": user_question})
print(smth)
output_string = smth.replace("```sql", "").replace("```", "")
print(output_string)

```
SELECT gaming_laptops.title, feature_gaming_laptops.feature_name, feature_gaming_laptops.feature_value
FROM gaming_laptops
JOIN gaming_laptop_feature ON gaming_laptops.id = gaming_laptop_feature.product_id
JOIN feature_gaming_laptops ON gaming_laptop_feature.feature_id = feature_gaming_laptops.id
WHERE feature_gaming_laptops.feature_name = 'screen_resolution'
  AND feature_gaming_laptops.feature_value = '1920 x 1080'
  AND EXISTS (
    SELECT 1
    FROM gaming_laptop_feature glf
    JOIN feature_gaming_laptops fgl ON glf.feature_id = fgl.id
    WHERE fgl.feature_name = 'storage_size_tb'
      AND glf.product_id = gaming_laptops.id
      AND fgl.feature_value = '1'
  );
```

SELECT gaming_laptops.title, feature_gaming_laptops.feature_name, feature_gaming_laptops.feature_value
FROM gaming_laptops
JOIN gaming_laptop_feature ON gaming_laptops.id = gaming_laptop_feature.product_id
JOIN feature_gaming_laptops ON gaming_laptop_feature.feature_id = feature_gaming_laptops.id
WHERE feature_g

In [14]:
sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

user_question = 'What gaming laptops are there that have a 1920x1080 display resolution and a 1TB SSD storage?'
smth= sql_chain.invoke({"features": features_string,"question": user_question})
print(smth)
output_string = smth.replace("```sql", "").replace("```", "")
print(output_string)


```sql
SELECT gaming_laptops.title, gaming_laptops.regular_price, gaming_laptops.happy_price, gaming_laptops.category
FROM gaming_laptops
JOIN gaming_laptop_feature ON gaming_laptops.id = gaming_laptop_feature.product_id
JOIN feature_gaming_laptops ON gaming_laptop_feature.feature_id = feature_gaming_laptops.id
WHERE gaming_laptops.category = 'gaming_laptopi'
AND feature_gaming_laptops.feature_name = 'screen_resolution'
AND feature_gaming_laptops.feature_value LIKE '%1920 x 1080%'
AND EXISTS (
    SELECT 1
    FROM gaming_laptop_feature
    JOIN feature_gaming_laptops ON gaming_laptop_feature.feature_id = feature_gaming_laptops.id
    WHERE gaming_laptops.id = gaming_laptop_feature.product_id
    AND feature_gaming_laptops.feature_name = 'storage_size_tb'
    AND feature_gaming_laptops.feature_value = '1'
)
```

This query will return the title, regular price, happy price, and category of gaming laptops that have a 1920x1080 display resolution and a 1TB SSD storage. The EXISTS subquer